# Homework 3
Culinary arts have evolved into an art form, with chefs creating unique dining experiences. Resources like the Michelin Guide are essential for discovering high-quality, innovative restaurants, catering to food enthusiasts, travelers, and industry professionals.
Our goal is to collect the information from the *Michelin Guide* in order to help users to find a restaurant that reflects their unique tastes.

- In order to do that, we firstly gather the data from the *Michelin Guide* [website](https://guide.michelin.com/en/it/restaurants) through web scraping techniques.

- Then, we build two types of search engines that allow users to retrieve restaurants according to their query, here the comparison is on the restaurant description. One that retrieves all the restaurants related to the query and another that ranks the results in terms of similarity.

- The next step is to base our search on all the different restaurant fields and therefore providing a new score.

- To further enhance the user experience, an interactive map will be developed. This feature will allow users to explore restaurants across Italy with customizable filters, such as price range and available services.

- Then, we focused on building an advanced search engine that retrieves restaurants according to 3 fields and enables the user to filter the visualized results.

In a world where cuisine is increasingly seen as an experience to be enjoyed and shared, this project represents an innovative approach to exploring Italy’s finest culinary offerings by merging technology with culinary culture.

In [5]:
# import all the necessary libraries
import requests
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep
import pandas as pd
import numpy as np
import csv
from collections import defaultdict
import math
from unidecode import unidecode
import heapq
from typing import List
import warnings
warnings.filterwarnings("ignore")

# Imports for Search Engine
import nltk  # Natural Language Toolkit for text processing
# Download necessary NLTK resources
nltk.download('stopwords')       # Stopwords list to filter out common words (e.g., "the", "is")
nltk.download('punkt')           # Punkt tokenizer for sentence splitting and word tokenization
nltk.download('wordnet')         # WordNet lexical database for lemmatization

# Importing specific modules for text processing
import re                         # Regular expressions for text pattern matching
from nltk.corpus import stopwords # Stopwords list for filtering out common, irrelevant words
from nltk.stem import WordNetLemmatizer # WordNet-based lemmatizer for reducing words to base form
from nltk.tokenize import word_tokenize # Word tokenizer to split text into individual words

# imports for map visalization and region/ coordinate collection
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


---
## 1. Collecting the data

### _Crawling_

We gather all restaurant URLs from the guide and store them in a text file for further data processing.

In [ ]:
# Header for the request (optional, can help prevent the server from blocking the request)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Open a file to write restaurant URLs
with open("michelin_restaurants_urls.txt", "w") as file:
    # Iterate through all pages (from 1 to 100)
    for page in range(1, 101):
        # Base URL with page number
        url = f"https://guide.michelin.com/en/it/restaurants/page/{page}"

        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Create the BeautifulSoup object to parse the HTML content
            soup = BeautifulSoup(response.text, "html.parser")

            # Find all restaurant links
            restaurant_links = soup.find_all("a", class_="link")

            # Iterate over restaurant links and write URLs to the file
            for link in restaurant_links:
                href = link.get("href")
                if href and "/restaurant/" in href:
                    full_url = f"https://guide.michelin.com{href}"
                    file.write(full_url + "\n")
                    print(f"URL added: {full_url}")
        else:
            print(f"Error in request to page {page}: {response.status_code}")

We download the HTML content of Michelin restaurant pages concurrently, organizing each batch of 20 URLs into separate folders with the restaurant's name as the filename.

In [ ]:
# Header for the request (optional)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Load URLs from the file
with open("michelin_restaurants_urls.txt", "r") as file:
    urls = [line.strip() for line in file.readlines()]

# Define the number of URLs per folder and max concurrent threads
url_per_folder = 20
max_threads = 5  # We can adjust the number of threads based on our network and system capacity

def download_html(url: str, folder_name: str) -> None:
    """
    Downloads the HTML content of a given URL and saves it as a file in the specified folder.
    The HTML file is named after the restaurant, based on the last part of the URL path.
    
    Parameters:
    -----------
    url : str
        The URL of the webpage to download.
    folder_name : str
        The folder path where the HTML file will be saved.
    
    Returns:
    --------
    None
    """
    try:
        # Extract restaurant name from the last part of the URL path
        restaurant_name = urlparse(url).path.split('/')[-1]
        
        # Send a GET request to retrieve the HTML content
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Define the file path and save HTML content to the file
            file_path = os.path.join(folder_name, f"{restaurant_name}.html")
            with open(file_path, "w", encoding="utf-8") as html_file:
                html_file.write(response.text)
            print(f"Saved: {file_path}")
        else:
            print(f"Error fetching {url}: {response.status_code}")
        
        sleep(0.1)  # Short delay to avoid overwhelming the server

    except requests.RequestException as e:
        # Log any network-related error that occurs during the request
        print(f"Error fetching {url}: {e}")

Saved: page_1\da-mo.html
Saved: page_1\o-me-o-il-mare.html
Saved: page_1\donevandro.html
Saved: page_1\da-bob-cook-fish.html
Saved: page_1\ape-vino-e-cucina.html
Saved: page_1\charleston.html
Saved: page_1\sa-domu-sarda.html
Saved: page_1\alessandro-feo.html
Saved: page_1\il-tirabuscio262517.html
Saved: page_1\la-buca130947.html
Saved: page_1\il-ristorante-alain-ducasse-napoli.html
Saved: page_1\dama-1213583.html
Saved: page_1\palazzo-utini.html
Saved: page_1\etra.html
Saved: page_1\soul-fish.html
Saved: page_1\la-trattoria-enrico-bartolini.html
Saved: page_1\loro.html
Saved: page_1\20tre.html
Saved: page_1\menage.html
Saved: page_1\procaccini.html
Saved: page_2\metodo-1213628.html
Saved: page_2\gimmy-s.html
Saved: page_2\osteria-dell-accademia.html
Saved: page_2\fratelli-bruzzone.html
Saved: page_2\serrae-villa-fiesole.html
Saved: page_2\locanda-perbellini-ai-beati.html
Saved: page_2\innesti.html
Saved: page_2\salvo.html
Saved: page_2\arnolfo.htmlSaved: page_2\osteria-del-teatro.html


In programming, threads allow multiple tasks to run concurrently within a single process, effectively enabling multitasking. A thread is a lightweight, independent sequence of execution within a program. Threads share the same memory space, making them more efficient for parallel tasks but also requiring careful handling of shared data to avoid conflicts.

This code downloads HTML content from a list of URLs in groups of 20 using threads for faster processing. It organizes the downloads into folders, with each folder containing a subset of URLs. For each group:

+ It calculates a folder name (like page_1, page_2) and creates it if it doesn’t already exist.
+ A subset of 20 URLs is selected.
+ A ThreadPoolExecutor is used to handle the concurrent downloads, where each URL is assigned to a thread.
+ It waits for all threads in the group to complete, retrieving any errors if they occur.
+ This approach improves efficiency by downloading multiple URLs concurrently in manageable groups.

In [ ]:
# Split URLs into groups of 20 and download concurrently
for i in range(0, len(urls), url_per_folder):
    folder_num = i // url_per_folder + 1
    folder_name = f"page_{folder_num}"
    
    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # Get the next 20 URLs
    url_subset = urls[i:i + url_per_folder]

    # Use ThreadPoolExecutor to download concurrently
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        # Start a thread for each URL in the subset
        futures = [executor.submit(download_html, url, folder_name) for url in url_subset]
        
        # Wait for all threads in the subset to complete
        for future in as_completed(futures):
            future.result()  # Retrieve any exceptions if occurred

### _Parsing_

We parse HTML files for Michelin restaurants to extract key information (e.g., name, address, cuisine, facilities), and save each restaurant's data as an individual .tsv file.

In [ ]:
def parse_restaurant_html(file_path: str) -> dict:
    """
    Parses an HTML file of a restaurant's webpage to extract details such as name, address, 
    cuisine type, and other amenities. Saves the extracted data to a .tsv file with the same 
    name as the input file and returns the data as a dictionary.

    Parameters:
    -----------
    file_path : str
        The path to the HTML file to parse.

    Returns:
    --------
    dict
        A dictionary containing extracted restaurant information.
    """
    
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Initialize dictionary with default empty strings for each field
    data = {
        "restaurantName": '',
        "address": '',
        "city": '',
        "postalCode": '',
        "country": "Italy",
        "priceRange": '',
        "cuisineType": '',
        "description": '',
        "facilitiesServices": '',
        "creditCards": '',
        "phoneNumber": '',
        "website": ''
    }

    # Extract restaurant name
    name_tag = soup.find("h1", class_="data-sheet__title")
    if name_tag:
        data["restaurantName"] = name_tag.get_text(strip=True)

    # Extract address information
    address_tag = soup.find_all("div", class_="data-sheet__block--text")
    if address_tag:
        full_address = address_tag[0].get_text(strip=True)
        address_parts = full_address.split(', ')
        if len(address_parts) >= 3:
            data["country"] = address_parts[-1]
            data["postalCode"] = address_parts[-2]
            data["city"] = address_parts[-3]
            data["address"] = ', '.join(address_parts[:-3])
        elif len(address_parts) == 2:
            data["address"], data["city"] = address_parts
        else:
            data["address"] = full_address

    # Extract price range and cuisine type
    if len(address_tag) > 1:
        full_price_type = address_tag[1].get_text(strip=True)
        price_type_list = full_price_type.split()
        if price_type_list:
            data["priceRange"] = price_type_list[0]
            data["cuisineType"] = ' '.join(price_type_list[2:])

    # Extract description
    description_tag = soup.find("div", class_="data-sheet__description")
    if description_tag:
        data["description"] = description_tag.get_text(strip=True)

    # Extract facilities and services
    facilities_section = soup.find('div', class_="restaurant-details__services")
    if facilities_section:
        services = [li.get_text(strip=True) for li in facilities_section.find_all('li')]
        data["facilitiesServices"] = ', '.join(services)

    # Extract accepted credit cards
    credit_card_tags = soup.find_all("div", class_="restaurant-details__services--info")
    creditCards = []
    for tag in credit_card_tags:
        for img in tag.find_all("img"):
            if 'data-src' in img.attrs:
                credit_card_name = os.path.basename(img['data-src']).split('-')[0]
                creditCards.append(credit_card_name.title())
    data["creditCards"] = ', '.join(creditCards) if creditCards else ''

    # Extract phone number
    phone_tag = soup.find("div", class_="collapse__block-item")
    if phone_tag:
        data["phoneNumber"] = phone_tag.get_text(strip=True)

    # Extract restaurant website
    div_web = soup.find('div', class_='collapse__block-item link-item')
    if div_web:
        web_tag = div_web.find('a', class_='link js-dtm-link')
        if web_tag and web_tag.get('href'):
            data["website"] = web_tag.get('href')

    # Save extracted data to a .tsv file
    output_file = file_path.replace(".html", ".tsv")
    with open(output_file, mode='w', encoding='utf-8') as file:
        keys = list(data.keys())
        dict_writer = csv.DictWriter(file, fieldnames=keys, delimiter='\t')
        dict_writer.writeheader()
        dict_writer.writerow(data)

    return data

Now, we process each restaurant's HTML file to extract data using concurrent threads, making the data collection process more efficient. Once all files are processed, we gather the generated .tsv files from each folder, read them into individual DataFrames, and then concatenate them into a single, unified DataFrame. Finally, we save this combined dataset to a single .tsv file, which consolidates all restaurant information for easy analysis and access.

In [ ]:
# Base directory containing the HTML files
base_directory = r"C:\Users\Utente\OneDrive - uniroma1.it\Esami\ADM\Homework 3"


def process_restaurant_file(file_info: tuple[str, str]) -> dict:
    """
    Processes a single HTML file to extract restaurant data.

    Given a tuple with the file name and folder path, this function constructs the full file path,
    parses the HTML to extract relevant restaurant information, and returns the data as a dictionary.

    Parameters:
    -----------
    file_info : tuple[str, str]
        A tuple containing the file name and the folder path where the HTML file is located.

    Returns:
    --------
    dict
        A dictionary containing extracted restaurant information.
    """
    filename, folder_path = file_info
    file_path = os.path.join(folder_path, filename)  # Construct full path to HTML file
    
    # Parse HTML file and extract restaurant data
    restaurant_data = parse_restaurant_html(file_path)
    
    return restaurant_data

# Collect all files to process
files_to_process = []
for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".html"):
                files_to_process.append((filename, folder_path))

# Set up concurrent processing
with ThreadPoolExecutor() as executor:
    # Schedule each file to be processed concurrently
    futures = {executor.submit(process_restaurant_file, file_info): file_info for file_info in files_to_process}

# Combine all created .tsv files into a single DataFrame

# List to collect each DataFrame
all_data = []

# Iterate over each folder in the base directory
for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    if os.path.isdir(folder_path):
        # Iterate over each .tsv file in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith(".tsv"):
                file_path = os.path.join(folder_path, filename)
                # Read each .tsv file and append to the list
                df = pd.read_csv(file_path, sep='\t',dtype={'postalCode': str}, keep_default_na=False) # Prevent reading NaN values and converting postal codes to strings
                all_data.append(df)

# Concatenate all individual DataFrames into one combined DataFrame
data = pd.concat(all_data, ignore_index=True)

We check if any missing values are present. In order to count the number the number of missing information, we need to count the number of empty strings.

In [ ]:
missing_values = (data == '').sum()
print(missing_values)

restaurantName          0
address                 0
city                    0
postalCode              0
country                 0
priceRange              0
cuisineType             0
description             0
facilitiesServices     24
creditCards             4
phoneNumber             0
website               108
text_to_compare         0
region                  0
dtype: int64


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   restaurantName      1982 non-null   object
 1   address             1982 non-null   object
 2   city                1982 non-null   object
 3   postalCode          1982 non-null   object
 4   country             1982 non-null   object
 5   priceRange          1982 non-null   object
 6   cuisineType         1982 non-null   object
 7   description         1982 non-null   object
 8   facilitiesServices  1982 non-null   object
 9   creditCards         1982 non-null   object
 10  phoneNumber         1982 non-null   object
 11  website             1982 non-null   object
 12  text_to_compare     1982 non-null   object
 13  region              1982 non-null   object
dtypes: object(14)
memory usage: 216.9+ KB


Then, we save in a tsv file the dataset just created.

In [283]:
# Export data to a tsv file
output_file = "michelin_restaurants_data.tsv"
data.to_csv(output_file, sep='\t', index=False)

---
## 2. Search Engine

### _Conjunctive Search Engine_

A conjunctive search engine retrieves documents that contain all specified search terms, meaning it only returns results where every query term appears. Our first version of the search engine is performed on the restaurant descriptions. The first step is to define a function that is going to process both the query and the description text, by lowercasing, removing non-alphanumeric characters, tokenizing into words, removing stopwords, and lemmatizing each word. Then, we build the dictionary and the inverted index that eventually will be used to perform the conjunctive search.

In [315]:
# Join the cuisine type and the description of the restaurant
data = pd.read_csv("michelin_restaurants_data.tsv", sep='\t', dtype={'postalCode': str}, keep_default_na=False)

Here, we define a function to preprocess text by cleaning, tokenizing, and normalizing it for our search engine. First, we convert text to lowercase, remove non-alphanumeric characters, and split it into individual words. We then remove stopwords and apply lemmatization to reduce each word to its base form, ensuring that our search engine captures the essential meaning of each term while discarding irrelevant details.

In [316]:
def preprocess_text(text: str) -> list[str]:
    """
    Preprocesses input text by converting it to lowercase, removing non-alphanumeric characters,
    tokenizing into words, removing stopwords, and lemmatizing each word.

    Parameters:
    -----------
    text : str
        The text to preprocess.

    Returns:
    --------
    list[str]
        A list of preprocessed tokens (words).
    """
    # Convert text to lowercase
    text = text.lower()
    
    # Remove any non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenize the text into individual words
    tokens = word_tokenize(text)
    
    # Remove common English stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Apply lemmatization to each word for standardization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

Now, we build a vocabulary and an inverted index to efficiently manage and search terms across restaurant descriptions. First, we assign a unique term ID to each word encountered, storing it in the vocabulary, and then we map each term ID to the document IDs where the term appears, creating the inverted index. 

In [317]:
def create_vocabulary_and_inverted_index(descriptions: list[str]) -> tuple[dict, dict]:
    """
    Creates a vocabulary and inverted index from a list of text descriptions.

    - **Vocabulary**: Maps each unique token to a unique term ID.
    - **Inverted Index**: Maps each term ID to a list of document IDs where the token appears.

    Parameters:
    -----------
    descriptions : list[str]
        A list of text descriptions, where each entry represents a document.

    Returns:
    --------
    tuple[dict, dict]
        A tuple containing:
        - vocabulary: dict mapping each unique token to a term ID.
        - inverted_index: dict mapping each term ID to a list of document IDs containing the term.
    """
    vocabulary = {}         # Dictionary to store unique words with their term IDs
    inverted_index = {}      # Dictionary to store the inverted index
    term_id = 0

    # Loop through each description, assigning a unique doc_id to each
    for doc_id, text in enumerate(descriptions):
        tokens = preprocess_text(text)  # Preprocess and tokenize the text

        for token in tokens:
            # Add the token to the vocabulary if it's not already there, assigning a new term ID
            if token not in vocabulary:
                vocabulary[token] = term_id
                term_id += 1

            # Get the term_id for the token
            tid = vocabulary[token]

            # Add doc_id to the inverted index for this term ID if not already added
            if tid not in inverted_index:
                inverted_index[tid] = []
            if doc_id not in inverted_index[tid]:
                inverted_index[tid].append(doc_id)

    return vocabulary, inverted_index

We implement a function to handle conjunctive queries, allowing us to search for restaurants that contain all specified terms in their descriptions. We process the query by mapping each search term to its unique term ID, then use the inverted index to retrieve only the documents (restaurants) containing all query terms. This approach ensures that our search engine provides precise results by returning only the restaurant data that fully matches the user’s search criteria.

In [318]:
def conjunctive_query(query: str, vocabulary: dict, inverted_index: dict, restaurant_data: pd.DataFrame) -> pd.DataFrame:
    """
    Performs a conjunctive search query, retrieving documents that contain all terms in the query.

    This function preprocesses the query, converts each term to a term ID using the vocabulary, 
    and uses the inverted index to identify documents that contain all query terms. It then 
    returns relevant information for the matching restaurants.

    Parameters:
    -----------
    query : str
        The search query as a string of terms.
    vocabulary : dict
        A dictionary mapping terms to unique term IDs.
    inverted_index : dict
        A dictionary mapping term IDs to sets of document IDs that contain the term.
    restaurant_data : pd.DataFrame
        The dataset containing restaurant information, indexed by document IDs.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the matching restaurant data with columns: 
        'restaurantName', 'address', 'description', 'website'.
    """
    # Preprocess the query text and convert it to term IDs using the vocabulary
    query_tokens = preprocess_text(query)
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]

    # If no terms from the query are found in the vocabulary, return an empty result
    if not query_term_ids:
        return pd.DataFrame(columns=['restaurantName', 'address', 'description', 'website'])

    # Find documents containing all term IDs from the query
    doc_lists = [set(inverted_index[tid]) for tid in query_term_ids if tid in inverted_index]

    # Intersect document lists to get documents that contain all query terms
    matching_docs = set.intersection(*doc_lists) if doc_lists else set()

    # Return the corresponding restaurant data for matching documents
    results_df = restaurant_data[restaurant_data.index.isin(matching_docs)]

    # Return just the key information (restaurant name, address, description, and website)
    return results_df[['restaurantName', 'address', 'description', 'website']]

We now test our search engine by running a sample query for "modern seasonal cuisine" to see if it accurately returns matching restaurants. We generate the vocabulary and inverted index from our dataset and use them to process the query, displaying the names and descriptions of any restaurants that meet the criteria. Finally, we summarize the results by indicating the total number of matches, which helps verify the effectiveness of our search engine in retrieving relevant data.

In [319]:
# Define the query to test the search engine
query = 'modern seasonal cuisine'

# Build the vocabulary and inverted index from the descriptions
vocabulary, inverted_index = create_vocabulary_and_inverted_index(data['description'])

# Run the conjunctive query
results = conjunctive_query(query, vocabulary, inverted_index, data)

# Display the results
print(f"Results for the query '{query}':")
results

Results for the query 'modern seasonal cuisine':


,restaurantName,address,description,website
26,Il Luogo Aimo e Nadia,via Montecuccoli 6,This long-established restaurant has been part...,https://www.aimoenadia.com/il-luogo-aimo-e-nadia
144,Ca' Del Moro,località Erbin 31,Situated within the La Collina dei Ciliegi win...,https://www.cadelmoro.wine/it
165,Contrasto,via Roma 55,"Having returned to his native village, owner-c...",https://contrastoristorante.it
178,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it
278,San Michele,via Castello di Fagagna 33,Situated next to the ruins of the old castle a...,http://sanmichele.restaurant
308,Chichibio,via Guglielmo Marconi 1,"Despite its lack of awards, this restaurant st...",
509,Esplanade,via Lario 3,"One of Italy’s long-established restaurants, t...",https://www.ristorante-esplanade.com/
513,La Valle,"via Umberto I 25, località Valle Sauglio",A well - run restaurant in a quiet area just o...,https://www.ristorantelavalle.it/
581,Zum Löwen,via Tirolo 25,A charming restaurant housed in a skilfully re...,
708,Degusteria del Gigante,via degli Anelli 19,Situated in the charming historic centre of th...,https://www.degusteriadelgigante.it/


The conjunctive search engine finds 31 restaurants whose cuisine is modern and seasonal. In this case, there is not an explicit order of the results as we designed a search engine that retrieves all the restaurants related to the query. 

### _Ranked Search Engine_
The engine evaluates the importance of each term in each document (restaurant information) using TF-IDF (Term Frequency - Inverse Document Frequency). TF measures how often a term appears in a document, while IDF reduces the weights (the importance) of terms that are present in many documents, prioritizing unique and informative terms.

In this cell, we calculate the Term Frequency-Inverse Document Frequency (TF-IDF) for each word across all restaurant descriptions to measure the relevance of each term within each document. First, we compute Term Frequency (TF) and Document Frequency (DF) for each term, then use these values to calculate TF-IDF scores, which help identify the most significant words in each description. Finally, we create an inverted index with TF-IDF weights, enabling efficient retrieval of relevant documents based on keyword importance in our search engine.

#### _Inverse Document Frequency (IDF)_

The IDF measures how unique or important a term \( t \) is across all documents:


$$ IDF(t) = \log \left( \frac{N}{DF(t) + \epsilon} \right) $$

Where:

- $N:$ Total number of documents in the dataset.
- $ DF(t) $: Document frequency, the number of documents containing the term \( t \).
- $ \epsilon $  : A small constant to avoid division by zero when  $DF(t) = 0$.

---

#### _TF-IDF Score_

The TF-IDF score for a term $t$ in a document $d$ is calculated as the product of its term frequency (TF) and inverse document frequency (IDF):

$$
TF\text{-}IDF(t, d) = TF(t, d) \cdot IDF(t)
$$

Where:

- $TF(t, d)$: Frequency of term $t$ in document $d$.
- $IDF(t)$: Inverse document frequency, as defined above.

---

#### _Normalization_

To ensure comparability of TF-IDF scores across documents, the TF-IDF vector for each document is normalized using the Euclidean norm:

$$
\text{Normalized TF-IDF}(t, d) = \frac{TF\text{-}IDF(t, d)}{\sqrt{\sum_{i \in d} (TF\text{-}IDF(i, d))^2} + \epsilon}
$$

Where:

- $t$: Term in the document d.
- $i$: Index for terms in document d.
- $\epsilon$    : A small constant added to avoid division by zero.


In [320]:
def calculate_tf_idf(data: pd.DataFrame, vocabulary: dict) -> dict:
    """
    Calculates the TF-IDF scores for each term in each document in the dataset and creates
    an inverted index with normalized TF-IDF weights.

    Parameters:
    -----------
    data : pd.DataFrame
        A DataFrame containing the documents, where each row is a document and the column
        'text_to_compare' contains the text to be processed.
    vocabulary : dict
        A dictionary mapping terms to unique term IDs.

    Returns:
    --------
    dict
        An inverted index where each term ID maps to a list of tuples, each containing a 
        document ID and the normalized TF-IDF score for that term in that document.
    """
    N = len(data)  # Total number of documents
    tf = defaultdict(lambda: defaultdict(int))  # Term Frequency (TF) for each document and term
    df = defaultdict(int)  # Document Frequency (DF) for each term
    tf_idf = defaultdict(lambda: defaultdict(float))  # TF-IDF scores for each document and term

    # Calculate TF and DF
    for doc_id, description in enumerate(data['description']):
        tokens = preprocess_text(description)  # Tokenize and preprocess the document text
        token_counts = defaultdict(int)
        
        for token in tokens:
            if token in vocabulary:
                token_id = vocabulary[token]
                token_counts[token_id] += 1
                tf[doc_id][token_id] = token_counts[token_id]

        # Increment DF for each unique token in the document
        for token_id in set(token_counts.keys()):
            df[token_id] += 1
    
    # Calculate TF-IDF for each term in each document
    for doc_id in tf.keys():
        for token_id, count in tf[doc_id].items():
            term_frequency = count
            epsilon = 1e-10  # Small constant to avoid division by zero
            inverse_document_frequency = math.log(N / (df[token_id] + epsilon))
            tf_idf[doc_id][token_id] = term_frequency * inverse_document_frequency

    # Normalize each document's TF-IDF vector and create an inverted index with normalized weights
    inverted_index = defaultdict(list)
    for doc_id, token_weights in tf_idf.items():
        doc_vector_norm = math.sqrt(sum(weight ** 2 for weight in token_weights.values()))
        for token_id, weight in token_weights.items():
            normalized_weight = weight / (doc_vector_norm + epsilon)  # Normalize term weight
            inverted_index[token_id].append((doc_id, normalized_weight))

    return inverted_index

Here, we implement a ranked search function that allows us to retrieve the top results based on similarity to a user query. We calculate TF-IDF scores for the query terms, then compute cosine similarity scores between the query and each document to rank the results. By selecting the highest-ranking matches, we ensure that our search engine returns the most relevant restaurant descriptions for each query.

In [321]:
def ranked_query(query: str, vocabulary: dict, inverted_index: dict, data: pd.DataFrame, top_k: int = 5) -> pd.DataFrame:
    """
    Performs a ranked search query, retrieving the top-k documents ranked by relevance to the query.

    This function preprocesses the query, calculates TF-IDF weights for each term in the query, 
    and computes the cosine similarity between the query and each document. It returns the top-k 
    most relevant documents based on the similarity score.

    Parameters:
    -----------
    query : str
        The search query as a string of terms.
    vocabulary : dict
        A dictionary mapping terms to unique term IDs.
    inverted_index : dict
        A dictionary mapping term IDs to lists of tuples, where each tuple contains a document ID
        and the TF-IDF weight of the term in that document.
    data : pd.DataFrame
        The dataset containing restaurant information, indexed by document IDs.
    top_k : int, optional
        The number of top results to return (default is 5).

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the top-k matching restaurant data, with columns: 
        'restaurantName', 'address', 'description', 'website', and 'similarity' score.
    """
    # Preprocess the query and convert terms to their IDs using the vocabulary
    query_tokens = preprocess_text(query)
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]
    
    # Calculate TF-IDF scores for the query terms
    query_tf_idf = defaultdict(float)
    for term_id in query_term_ids:
        if term_id is not None:
            query_tf_idf[term_id] += 1  # Term Frequency (TF) for query terms

    # Normalize the query TF-IDF vector to unit length
    query_vector_norm = math.sqrt(sum(weight ** 2 for weight in query_tf_idf.values()))
    query_tf_idf = {term_id: weight / (query_vector_norm + 1e-10) for term_id, weight in query_tf_idf.items()}

    # Calculate cosine similarity between the query and document vectors
    doc_scores = defaultdict(float)
    for term_id, query_weight in query_tf_idf.items():
        if term_id in inverted_index:
            for doc_id, doc_weight in inverted_index[term_id]:
                doc_scores[doc_id] += query_weight * doc_weight  # Accumulate similarity score for each document

    # Sort documents by similarity score in descending order and select the top-k results
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # Retrieve and format the top-k results with relevant restaurant information
    results = []
    for doc_id, score in ranked_docs:
        row = data.iloc[doc_id]
        results.append({
            "restaurantName": row['restaurantName'],
            "address": row['address'],
            "description": row['description'],
            "website": row['website'],
            "similarity": round(score, 3)  # Round similarity score to 3 decimal places
        })

    return pd.DataFrame(results)

In this cell, we calculate the TF-IDF-based inverted index, which allows us to weight terms by their importance across all restaurant descriptions. We then test our search engine by running a ranked query for "modern seasonal cuisine," retrieving and displaying the top five most relevant restaurant results. This step helps us evaluate the search engine’s effectiveness in returning high-quality, relevant matches based on user input.

In [322]:
# Calculate the inverted index with TF-IDF weights
inverted_index_tf_idf = calculate_tf_idf(data, vocabulary)

# Perform a ranked query
query = 'modern seasonal cuisine'
results = ranked_query(query, vocabulary, inverted_index_tf_idf, data, top_k=5)

# Display the results for the query
print(f"Results for the ranked query '{query}':")
results

Results for the ranked query 'modern seasonal cuisine':


,restaurantName,address,description,website,similarity
0,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, al...",https://ristorantesaur.it,0.233
1,La Botte,via Giuseppe Garibaldi 8,A modern and welcoming contemporary bistro sit...,http://www.trattorialabottestresa.it,0.208
2,Piccolo Lord,corso San Maurizio 69 bis/g,"Professional service in a welcoming, modern re...",https://www.ristorantepiccololord.it/,0.204
3,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/,0.200
4,La Valle,"via Umberto I 25, località Valle Sauglio",A well - run restaurant in a quiet area just o...,https://www.ristorantelavalle.it/,0.181


The restaurant Saur ranks highest with a similarity score of 0.2329, reflecting its close alignment with the user's request for "modern seasonal cuisine." The scores progressively decrease, with La Valle being the fifth-ranked result. This output highlights the engine's ability to prioritize relevance using advanced natural language processing techniques such as TF-IDF weighting and cosine similarity, ensuring an intuitive and effective search experience for users.

---
## 3. Define a New Score

We will now enhance restaurant rankings by introducing a custom scoring metric that combines description match (TF-IDF), cuisine type, facilities/services, and price range. Using a heap for efficient ranking, we will prioritize restaurants based on user preferences. The output will display key details and a new similarity score. Finally, we’ll compare results to assess improvements over the previous method.

### _Data Preprocessing for Enhanced Search_

In this cell, we preprocess the **`priceRange`**, **`creditCards`**, and **`facilitiesServices`** columns in the dataset:


1. **Clean Credit Card Information**: Remove unnecessary characters (e.g., quotes, brackets, commas) from the **`creditCards`** column to make it easier to process and match during filtering.

2. **Clean Facilities/Services**: Similarly, clean the **`facilitiesServices`** column to standardize and prepare it for efficient filtering.

This ensures the dataset is better aligned with the search engine’s scoring and filtering mechanisms.


In [323]:
data['creditCards_converted'] = data['creditCards'].str.replace(r"[\"\'\[\],]", '', regex=True).str.strip()

data['facilitiesServices_converted'] = data['facilitiesServices'].str.replace(r"[\"\'\[\],]", '', regex=True).str.strip()

### _Creating a Searchable Text Field_

In this cell, we combine multiple fields (e.g., restaurant name, address, city, country, cuisine type, and more) into a single column, **`text_to_compare`**, to create a comprehensive searchable text field. 

- **Unidecode Conversion**: Convert special characters (e.g., `é`, `á`) to their simpler forms (`e`, `a`) for generalization and easier search.
- **Field Combination**: Concatenate relevant columns to ensure that the conjunctive search retrieves all relevant information across multiple attributes.

This step ensures the search engine can process diverse user queries effectively.


In [324]:
data['text_to_compare'] = data['restaurantName'].apply(unidecode) + ' '+data['address'] + ' '+data['city'] + ' '+data['country'] + ' '+data['cuisineType'] + ' '+ data['description'] + ' '+data['facilitiesServices'].astype(str) + ' ' + data['creditCards_converted'].astype(str)+' ' + data['postalCode'].astype(str) + " " + data['priceRange']

### _Running the Conjunctive Query_

- **Vocabulary and Inverted Index**: Generate a vocabulary and inverted index from the combined `text_to_compare` field.
- **Query Execution**: Perform a conjunctive query to retrieve documents matching the query terms.
- **Reindexing Results**: Reset the index of the filtered results for streamlined processing in subsequent steps.


In [325]:
vocabulary, inverted_index = create_vocabulary_and_inverted_index(data['text_to_compare'])

# Run the conjunctive query
relevant_documents = conjunctive_query(query, vocabulary, inverted_index, data)
relevant_documents = relevant_documents.reset_index(drop=True) #here we reordered the index as we will only be working with these documents.

In [ ]:
relevant_documents.hesd()

### _Scoring Documents with TF-IDF_

- **Preprocessing**: Tokenize and preprocess the query, filtering terms if specified.
- **TF-IDF Calculation**: Compute term frequencies for the query, normalize the vector, and calculate cosine similarity with the documents.
- **Ranking**: Rank documents based on their similarity scores.
- **Output**: Return a DataFrame containing detailed restaurant information along with the calculated scores.


In [326]:
def score_with_tfIdf(
    query: str, 
    vocabulary: Dict[str, int], 
    inverted_index: Dict[int, list], 
    data: pd.DataFrame, 
    remove_words_from_query: bool = False
) -> pd.DataFrame:
    """
    Scores and ranks documents using TF-IDF and cosine similarity.

    This function preprocesses the user query, calculates TF-IDF scores for the query, 
    and computes cosine similarity between the query vector and document vectors in the dataset. 
    The results are returned as a ranked DataFrame containing restaurant details and scores.

    Parameters:
    -----------
    query : str
        The user query containing terms to search for.
    vocabulary : Dict[str, int]
        A dictionary mapping terms to unique term IDs.
    inverted_index : Dict[int, list]
        An inverted index mapping term IDs to lists of tuples, where each tuple contains a document ID and TF-IDF weight.
    data : pd.DataFrame
        The complete dataset containing restaurant information.
    remove_words_from_query : bool, optional
        If True, removes query terms not found in the vocabulary, by default False.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing ranked documents with their scores and relevant restaurant details.

    Steps:
    ------
    1. **Query Preprocessing**: Tokenize and preprocess the query text.
    2. **TF-IDF Calculation**: Compute term frequencies for the query, normalize the query vector, 
       and calculate cosine similarity between the query vector and document vectors.
    3. **Document Ranking**: Rank documents based on their similarity scores.
    4. **Output**: Return a DataFrame with restaurant details and scores.

    Example:
    --------
    results = score_with_tfIdf("seafood Italian", vocabulary, inverted_index, data, remove_words_from_query=True)
    print(results)
    """
    # Preprocess the query and optionally filter out terms not in the vocabulary
    query_tokens = preprocess_text(query)
    if remove_words_from_query:
        query_tokens = [token for token in query_tokens if token in vocabulary]
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]
    
    # Calculate TF-IDF scores for the query
    query_tf_idf = defaultdict(float)
    for term_id in query_term_ids:
        if term_id:
            query_tf_idf[term_id] += 1  # Increment Term Frequency (TF) for each term

    # Normalize the query TF-IDF vector
    query_vector_norm = math.sqrt(sum(weight ** 2 for weight in query_tf_idf.values()))
    query_tf_idf = {term_id: weight / (query_vector_norm + 1e-10) for term_id, weight in query_tf_idf.items()}

    # Calculate cosine similarity between query vector and document vectors
    doc_scores = defaultdict(float)
    for term_id, query_weight in query_tf_idf.items():
        if term_id in inverted_index:
            for doc_id, doc_weight in inverted_index[term_id]:
                doc_scores[doc_id] += query_weight * doc_weight  # Accumulate similarity scores

    # Sort documents by their similarity scores in descending order
    sorted_Documents = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Prepare the DataFrame to store ranked documents
    sorted_Documents_dataFrame = []
    for doc_id, score in sorted_Documents:
        row = data.iloc[doc_id]
        sorted_Documents_dataFrame.append({
            "restaurantName": row['restaurantName'],
            "address": row['address'],
            "city": row['city'],
            "postalCode": row['postalCode'],
            "country": row['country'],
            "priceRange": row['priceRange'],
            "cuisineType": row['cuisineType'],
            "description": row['description'],
            "facilitiesServices": row['facilitiesServices'],
            "creditCards": row['creditCards'],
            "phoneNumber": row['phoneNumber'],
            "website": row['website'],
            # "priceRange_maped": row['priceRange_maped'],
            "creditCards_converted": row['creditCards_converted'],
            "facilitiesServices_converted": row['facilitiesServices_converted'],
            "score": score  # Similarity score between query and document
        })

    # Return the ranked DataFrame
    return pd.DataFrame(sorted_Documents_dataFrame)

In this cell, we calculate TF-IDF scores specifically for the restaurant descriptions. We create a vocabulary and inverted index for the description field and then compute similarity scores between the user query and the descriptions using the score_with_tfIdf function. The output displays the top-ranked results based on this similarity.

In [327]:
#to calculate tf-idf score with description
inverted_index_tf_idf2 = calculate_tf_idf(relevant_documents, vocabulary)
vocabulary2, inverted_index2 = create_vocabulary_and_inverted_index(relevant_documents['description'])
score_from_description = score_with_tfIdf(query, vocabulary2, inverted_index_tf_idf2, data)
print(score_from_description.shape)
score_from_description.head(2)


(27, 15)


,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,creditCards_converted,facilitiesServices_converted,score
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,Air conditioning,"Amex, Dinersclub, Mastercard, Visa",+39 010 247 6191,https://www.ristorante20tregenova.it/,Amex Dinersclub Mastercard Visa,Air conditioning,0.304558
1,Luminist Cafè Bistrot,via Toledo 177,Naples,80134,Italy,€€,Classic Cuisine,Situated on the ground floor of the splendid B...,"Air conditioning, Wheelchair access","Amex, Unionpay, Dinersclub, Discover, Jcb, Mae...",+39 081 1975 8023,http://www.giuseppeiannotti.it/Luminist,Amex Unionpay Dinersclub Discover Jcb Maestroc...,Air conditioning Wheelchair access,0.121884


### _Cuisine Match, Facilities and Services and Price Range_ 
This function is used to find matches with the particular parameter and increase the score accordingly. As the goal is to find top retaurents and not similarity the score is not bound to any top rangents.
- **Ranking**: Rank documents based on their similarity scores.
- **Output**: Return a DataFrame containing detailed restaurant information along with the calculated scores.


In [328]:
def score_add(
    query: str, 
    vocabulary: Dict[str, int], 
    inverted_index: Dict[int, list], 
    restaurant_data: pd.DataFrame, 
    point_to_add: float
) -> pd.DataFrame:
    """
    Adjusts the scores of documents based on query term matches.

    This function preprocesses the user query, identifies matching terms in the vocabulary,
    and updates the scores of the corresponding documents in the dataset.

    Parameters:
    -----------
    query : str
        The user query containing terms to search for.
    vocabulary : Dict[str, int]
        A dictionary mapping terms to unique term IDs.
    inverted_index : Dict[int, list]
        An inverted index mapping term IDs to lists of document IDs where the term appears.
    restaurant_data : pd.DataFrame
        A DataFrame containing restaurant information and their current scores.
    point_to_add : float
        The number of points to add to the score of each document that contains a matching term.

    Returns:
    --------
    pd.DataFrame
        A DataFrame with updated scores for documents containing query terms.

    Steps:
    ------
    1. **Query Preprocessing**: Tokenize and preprocess the query text.
    2. **Term Matching**: Find term IDs corresponding to the query tokens in the vocabulary.
    3. **Score Update**: For each matching term ID, increment the scores of relevant documents.
    4. **Output**: Return the updated DataFrame with modified scores.

    """
    # Preprocess the query to extract tokens
    query_tokens = preprocess_text(query)

    # Map query tokens to term IDs using the vocabulary
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]

    # Update scores for documents containing matching term IDs
    for term_id in query_term_ids:
        if term_id in inverted_index:
            matching_rows = inverted_index[term_id]  # Get document IDs from the inverted index
            
            # Increment the score for each matching document
            restaurant_data.loc[matching_rows, 'score'] += point_to_add

    # Return the updated DataFrame
    return restaurant_data


### _Cuisine Match: Increase the score for matching cuisine types._
- **vocabulary_c**: This is the vocabulary that consist only considering cuisineType
- **inverted_index_c**: This is the inverted index that consist only considering cuisineType. It maps the vocabulary words that is found in which documnets or records of the dataframe. marking the indexes
- **scored**: This function return a data frame with score increased following the matches


In [329]:
#for cuisineType
vocabulary_c, inverted_index_c = create_vocabulary_and_inverted_index(score_from_description['cuisineType']) #vocab with only cuisineType as we score only focusing cuisibe type


scored = score_add(query, vocabulary_c, inverted_index_c, score_from_description, 0.1) #0.1 is the point to increase for match


### _Facilities and Services: Give more points for matching facilities/services (e.g., “Terrace,” “Air conditioning”)._
- **vocabulary_f**: This is the vocabulary that consist only considering facilitiesServices_converted
- **inverted_index_f**: This is the inverted index that consist only considering facilitiesServices_converted. It maps the vocabulary words that is found in which documnets or records of the dataframe. marking the indexes
- **scored**: This function return a data frame with score increased following the matches


In [330]:
#for facilitiesServices
vocabulary_f, inverted_index_f = create_vocabulary_and_inverted_index(scored['facilitiesServices_converted']) #vocab with only facilities and scoring only based on facilities
scored = score_add(query, vocabulary_f, inverted_index_f, scored, 0.2)

In [331]:
scored.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,creditCards_converted,facilitiesServices_converted,score
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,Air conditioning,"Amex, Dinersclub, Mastercard, Visa",+39 010 247 6191,https://www.ristorante20tregenova.it/,Amex Dinersclub Mastercard Visa,Air conditioning,0.504558
1,Luminist Cafè Bistrot,via Toledo 177,Naples,80134,Italy,€€,Classic Cuisine,Situated on the ground floor of the splendid B...,"Air conditioning, Wheelchair access","Amex, Unionpay, Dinersclub, Discover, Jcb, Mae...",+39 081 1975 8023,http://www.giuseppeiannotti.it/Luminist,Amex Unionpay Dinersclub Discover Jcb Maestroc...,Air conditioning Wheelchair access,0.221884
2,Il Ristorante - Niko Romito,via di Ripetta 73,Rome,186,Italy,€€€€,Italian Contemporary,Situated on the fifth floor of the Hotel Bulga...,"Air conditioning, Interesting wine list, Terrace","Amex, Unionpay, Dinersclub, Discover, Jcb, Mae...",+39 06 3608 0410,https://www.bulgarihotels.com/it_IT/rome/dinin...,Amex Unionpay Dinersclub Discover Jcb Maestroc...,Air conditioning Interesting wine list Terrace,0.090678
3,Quintogusto,piazza Sandro Pertini 54 r,Savona,17100,Italy,€€,"Contemporary, Mediterranean Cuisine",Recently opened in the renovated section of Co...,"Air conditioning, Terrace, Wheelchair access","Amex, Mastercard, Visa",+39 019 770 2870,https://ristorantequintogusto.com/,Amex Mastercard Visa,Air conditioning Terrace Wheelchair access,0.190014
4,LoRo,via Bruse 2,Trescore Balneario,24069,Italy,€€€,Creative,The cuisine served at LoRo is imaginative both...,"Air conditioning, Car park","Amex, Maestrocard, Mastercard, Visa",+39 347 761 4728,http://www.loroandco.com,Amex Maestrocard Mastercard Visa,Air conditioning Car park,0.086054


### _Price Range: Higher scores could be given to more affordable options based on the user’s choice._
- **price_mapping**: it consists of the words that can be found in dataframe for priceRange_maped.
- **vocabulary_p**: This is the vocabulary that consist only considering priceRange_maped
- **inverted_index_p**: This is the inverted index that consist only considering priceRange_maped. It maps the vocabulary words that is found in which documnets or records of the dataframe. marking the indexes
- **scored**: This function return a data frame with score increased following the matches


In [332]:
#price_mapping
price_mapping = [
    '€',
    '€€',
    '€€€',
    '€€€€'
]
for i in range(len(price_mapping)):
    vocabulary_p, temp = create_vocabulary_and_inverted_index(price_mapping[i])
    temp, inverted_index_p = create_vocabulary_and_inverted_index(scored['priceRange'])
    scored = score_add(price_mapping[i], vocabulary_p, inverted_index_p, scored, 0.025*(4-i))
    


In [333]:
scored.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,creditCards_converted,facilitiesServices_converted,score
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,Air conditioning,"Amex, Dinersclub, Mastercard, Visa",+39 010 247 6191,https://www.ristorante20tregenova.it/,Amex Dinersclub Mastercard Visa,Air conditioning,0.504558
1,Luminist Cafè Bistrot,via Toledo 177,Naples,80134,Italy,€€,Classic Cuisine,Situated on the ground floor of the splendid B...,"Air conditioning, Wheelchair access","Amex, Unionpay, Dinersclub, Discover, Jcb, Mae...",+39 081 1975 8023,http://www.giuseppeiannotti.it/Luminist,Amex Unionpay Dinersclub Discover Jcb Maestroc...,Air conditioning Wheelchair access,0.221884
2,Il Ristorante - Niko Romito,via di Ripetta 73,Rome,186,Italy,€€€€,Italian Contemporary,Situated on the fifth floor of the Hotel Bulga...,"Air conditioning, Interesting wine list, Terrace","Amex, Unionpay, Dinersclub, Discover, Jcb, Mae...",+39 06 3608 0410,https://www.bulgarihotels.com/it_IT/rome/dinin...,Amex Unionpay Dinersclub Discover Jcb Maestroc...,Air conditioning Interesting wine list Terrace,0.090678
3,Quintogusto,piazza Sandro Pertini 54 r,Savona,17100,Italy,€€,"Contemporary, Mediterranean Cuisine",Recently opened in the renovated section of Co...,"Air conditioning, Terrace, Wheelchair access","Amex, Mastercard, Visa",+39 019 770 2870,https://ristorantequintogusto.com/,Amex Mastercard Visa,Air conditioning Terrace Wheelchair access,0.190014
4,LoRo,via Bruse 2,Trescore Balneario,24069,Italy,€€€,Creative,The cuisine served at LoRo is imaginative both...,"Air conditioning, Car park","Amex, Maestrocard, Mastercard, Visa",+39 347 761 4728,http://www.loroandco.com,Amex Maestrocard Mastercard Visa,Air conditioning Car park,0.086054


In [334]:
#sorted with heapq
k = 5  

top_k = heapq.nlargest(k, scored.itertuples(index=False), key=lambda x: x.score) 




In [335]:
#desired output with columns maped from heap structure
desired_columns = ['restaurantName', 'address', 'description', 'website', 'score']
top_k_data = pd.DataFrame(top_k, columns=scored.columns)[desired_columns]
top_k_data 

,restaurantName,address,description,website,score
0,20Tre,via David Chiossone 20 r,Situated in the heart of Genoa’s historic cent...,https://www.ristorante20tregenova.it/,0.504558
1,Osteria Il Bagatto,via XX Settembre 16,"In a typical mountain - style ambience, enjoy ...",https://www.releven11.it/,0.285136
2,Locanda delle Tre Chiavi,via Vannetti 8,This restaurant housed in an 18C building is r...,http://www.locandadelletrechiavi.it,0.238958
3,La Magnolia,viale Morin 46,"Housed within the Lord Byron hotel, one of For...",https://www.lamagnoliaristorante.com/it/,0.234010
4,Casa Rispoli,piazza san Francesco 7,Overlooking a piazza surrounded by picturesque...,http://www.casarispoli.it,0.233029


### _Are the results you obtain better than with the previous scoring function? Explain and compare results._

In [336]:
i = 0
updated_score_description = []
for desc in top_k_data['description']:
    updated_score_description.append(desc)
    if i == 3:
        break
    i+=1

In [337]:
# # Perform a ranked query
inverted_index_tf_idf = calculate_tf_idf(data, vocabulary)


results = ranked_query(query, vocabulary, inverted_index_tf_idf, data, top_k=5)


In [338]:
i = 0
ranked_query_results = []
for desc in results['description']:
    ranked_query_results.append(desc)
    if i == 3:
        break
    i+=1

In [339]:
print("Query: ",query)
print()
for i in range(3):
    print("Question 3 new score: ",updated_score_description[i])
    print("------")
    print("Ranked Query: ",ranked_query_results[i])
    print("*********************************************")
    

Query:  modern seasonal cuisine

Question 3 new score:  Situated in the heart of Genoa’s historic centre, this contemporary-style restaurant focuses on just a few dishes, almost all fish-based, presented in a very modern style and in generous portions. Seasonal ingredients and market-fresh produce are the guiding philosophy here.
------
Ranked Query:  In a tiny rural village, this contemporary, almost minimalist-style restaurant serves modern cuisine with an emphasis on seasonal, regional produce.
*********************************************
Question 3 new score:  In a typical mountain - style ambience, enjoy carefully prepared cuisine made by a skilful chef using the best ingredients. The menu includes a good choice of regional specialities, as well as fish dishes and imaginative contemporary fare.
------
Ranked Query:  A modern and welcoming contemporary bistro situated in the heart of Stresa’s historic centre. Run by an entire family, the restaurant serves modern and imaginative fi

### _Explanation_
Based on the criterias of facilities, cuisine type and lower price range, the updated scoring system of question 3 works better. As it considers all this 3 expect separetly. 

More over, as the conjuctive search cancels out posibilities without the query key words and then tf-idf with cosine from description makes it easier to score accordingly. Thus conjunctive search reduces noices as it works as a filter removing unrelated documents. The primary reasons as it works better are:

1. Conjunctive search makes sure- that only documents containing all required terms are considered.
2. TF-IDF with cosine similarity then ranks these documents based on how similar they are to the query as a whole.
3. The cosine similarity scores based on the vector representation of the query and documents. It not only captures the presence of terms but also their relative importance and distribution across the query and documents.

Thus, by combining conjunctive filtering with Tf-IDF cosine similarity, we can focu only on documents that meet all query conditions and then rank them effectively.


On the other hand,
There are some drawbacks as in some cases this conjuctive search will ignore similar words or documents with same meaning b=using different words. As it makes the search really strict. Which is more flexible in Ranked Query.

---
## 4. Restaurant Visualization

In [2]:
# getting a dataframe from .tsv file created earlier after crawling and parsing
df = pd.read_csv('michelin_restaurants_data.tsv',sep = '\t') 

### _Fetching Regions_

We first find regions for each unique city using the `getRegion()` function that employs the use of **OpenStreetMap** service **Nominatim**. We find the regions based on _city, country_ first, if we don't find it then we request based on _city_ only and if still we don't get the required results, we request based on _postal code_.  
Then the regions for each city is mapped on to the main dataframe **df** based on each city. This column will be used further on for fetching the coordinates the resultant restaurants of our custom search and making our advanced search engine.

In [3]:
def getRegion(fullAddress, city, postalCode):

    # Initializing geocoder with Nominatim and user-agent for calling service
    geolocator = Nominatim(user_agent="getRegion")
    # Initializing RateLimiter to ensure minimum delay between geocoding requests
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.5)

    # getting region using the fullAddress i.e. City + Country
    location = geocode(fullAddress, addressdetails=True)
    if location and location.raw:
        address = location.raw.get("address", {})
        # the api returns some regions under the key 'region' and others under 'state'
        region = address.get("region") or address.get("state") 
        return pd.Series([region])
    else:
        # getting region using just the city name
        location = geocode(city, addressdetails=True)
        if location and location.raw:
            address = location.raw.get("address", {})
            region = address.get("region") or address.get("state")
            return pd.Series([region])
        else:
            # getting region using the postal code
            location = geocode(postalCode, addressdetails=True)
            if location and location.raw:
                address = location.raw.get("address", {})
                region = address.get("region") or address.get("state")
                return pd.Series([region])
    # if no region is found, return none
    return None

In [4]:
# making a dataframe with unique cities and required details
df_cities = df.drop_duplicates(subset=["city"])[["city", "country", "postalCode"]].reset_index(drop=True)
# creating column fullAddress using city + country
df_cities['fullAddress'] = df_cities['city'] + ", " + df_cities['country']

# getting region for each city
df_cities["region"] = df_cities.apply(lambda x: getRegion(x["fullAddress"], x["city"], x["postalCode"]), axis=1)
# mapping region column onto the main dataframe based on each city
df = df.merge(df_cities[['city','region']], on='city', how='left')

# saving the dataframe as a .tsv file
output_file = 'all_restaurants_data_regions.tsv'
df.to_csv(output_file, sep='\t', index=False, header=True)

### _Location Geocoding_

We will be performing geocoding of our restaurant locations using **Nominatim** which is a service provided by **OpenStreetMap**.  
The function `getCoordinates` gets full address, city name and postal code as input. It then uses the full address which is a combination of city name and country name i.e. Italy to get location to our specific city. The coordinates will be collected at run time, each time a new custom search is made.     
However, in particular cases, the API does not return the coordinates but it does in when only the city name is sent as a request. Furthermore, in a very few instances, it does not even return coordinates as it cannot identify the city so sending the postal code does the trick. This may be because the OpenStreetMap API is mainly reliant on community-driven data making it not as exhaustive.  

In [5]:
def getCoordinates(fullAddress: str, city: str, postalCode: int):
    
    # Initializing geocoder with Nominatim and user-agent for calling service
    geolocator = Nominatim(user_agent="getCoordinates")
    # Initializing RateLimiter to ensure minimum delay between geocoding requests
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.5)

    # getting coordinates using the fullAddress i.e. city + region
    coordinates = geocode(fullAddress)
    if coordinates:
        return pd.Series([coordinates.latitude, coordinates.longitude])
    else:
        # getting coordinates using just the city name
        coordinates = geocode(city)
        if coordinates:
            return pd.Series([coordinates.latitude, coordinates.longitude])
        else:
            # getting coordinates using the postal code
            coordinates = geocode(postalCode)
            if coordinates:
                return pd.Series([coordinates.latitude, coordinates.longitude])
            else:
                # if no coordiates are found return NONE
                return pd.Series([None, None])

### _Dataframe Compilation_

The `get_dataframe()` function takes in the dataframe that has the results from our custom search query and then matches it with the dataframe collected from the .tsv file (created as a result of data crawling and parsing) so that we can also get the relevant columns that will be needed for geocoding and restaurant mapping.  
The function then creates and extra column by the name of `fullAddress` consisting of `city` + `country` of that restaurant.  
We finally make two columns `latitude` and `longitude` to store coordinate data for each restaurant as a result of calling the `getCoordinates()` function.

In [33]:
def get_dataframe(top_k_data: pd.DataFrame):
    # making dataframe using data of all restaurants stored in .tsv file
    df = pd.read_csv('all_restaurants_data_regions.tsv',sep = '\t') 
    # finding relevant restaurants from the .tsv dataframe using merge
    df_location = top_k_data.merge(df, 
                on=['restaurantName', 'address'], 
                how='left')[['restaurantName', 'city', 'country', 'priceRange', 'postalCode' ,'region']]

    # creating column for full address
    df_location['fullAddress'] = df_location['city'] + ", " + df_location['region']
 
    # getting latitude and longitude for each restaurant
    df_location[["latitude", "longitude"]] = df_location.apply(lambda x: getCoordinates(x["fullAddress"], x["city"], x["postalCode"]), axis=1)
    
    return df_location


### _Unique Coordinates_

There may be many restaurants with the same coordinates because they are located in the same city. This will make one of the restaurant markers invisible to the naked eye on the map if there is already another marker in the same city.  
To avoid this problem, we randomly offset the restaurant coordinates that are non-unique upto **0.005** and then replace the new offset coordinates with the old duplicate coordinates in the function `coordinates_offset()`.

In [7]:
def coordinates_offset(df_location: pd.DataFrame):
    coordsExist = set()
    maxOffset = 0.005 # maximum offset for coordinates
    
    for index, row in df_location.iterrows():
        coords = (row['latitude'], row['longitude'])
    
        if coords in coordsExist:
            # duplicate coordinates are applied with a random offset
            latitudeOffset = row['latitude'] + np.random.uniform(-maxOffset, maxOffset)
            longitudeOffset = row['longitude'] + np.random.uniform(-maxOffset, maxOffset)
            df_location.at[index, 'latitude'] = latitudeOffset
            df_location.at[index, 'longitude'] = longitudeOffset
        else:
            # unique coordinates added to a set
            coordsExist.add(coords)

    return df_location

### _Setting Up Map_

We will display the restaurants on a map using **Folium**.  
In the function `mapSetup()`, initially a base map is created centered and zoomed in on Italy. Then a colour scheme is defined based on varying price ranges to differentiate between restaurant markers. To facilitate the user, a legend for the colour scheme made using HTML will be displayed on the bottom left corner of the map.  
Moreover, each marker will show the restaurant name and the city it is located in when clicked on it.

In [8]:
def mapSetup(df_location: pd.DataFrame):
    # creating base folim map centered on Italy
    italian_restaurants_map = folium.Map(location=[41.8719, 12.5674], tiles="Cartodb Positron", zoom_start=6)

    # price range colour schem defined
    colorScheme = {
        "€": "lightgreen",
        "€€": "blue",
        "€€€": "orange",
        "€€€€": "red"
    }

    # HTML Legend for price range colour scheme
    legend = """
         <div style="position: fixed; 
                     bottom: 30px; left: 30px; width: 150px; height: 130px; 
                     background-color: white; border:2px solid grey; z-index:9999; font-size:14px; padding:10px;">
             <b>Price Range</b><br>
             <i style="background-color:#b7f270; width:18px; height:18px; display:inline-block; margin-right:5px;"></i> €<br>
             <i style="background-color:#36a6d9; width:18px; height:18px; display:inline-block; margin-right:5px;"></i> €€<br>
             <i style="background-color:#f0932e; width:18px; height:18px; display:inline-block; margin-right:5px;"></i> €€€<br>
             <i style="background-color:#cd3b27; width:18px; height:18px; display:inline-block; margin-right:5px;"></i> €€€€<br>
         </div>
    """

    # adding markers on map for each restaurant
    for index, row in df_location.iterrows():
        popupData = f"<b>{row['restaurantName']}</b>---<i>City: {row['city']}</i>"
        folium.Marker(
            location = [row["latitude"], row["longitude"]],
            popup = folium.Popup(popupData, max_width=300),
            icon = folium.Icon(color=colorScheme.get(row["priceRange"], "gray"))
        ).add_to(italian_restaurants_map)

    italian_restaurants_map.get_root().html.add_child(folium.Element(legend)) #adding legend to the map
    italian_restaurants_map.save("italian_restaurants_map.html") # saving map as an interactive HTML file

    return italian_restaurants_map

### _Map Display_

The `get_restaurant_map()` function will call `get_dataframe()`, `coordinates_offset()` and `mapSetup()` functions respectively and return the map with markers for the respective restaurants at their coordinates, that we get as a result of our custom search engine.  
Moreover, the map is also saved onto an interactive HTML file so that it can be viewed in case it cannot be in the notebook.

In [9]:
def get_restaurants_map(top_k_data: pd.DataFrame):

    df_location = get_dataframe(top_k_data) # getting restaurant data with respective coordinates
    df_location = coordinates_offset(df_location) # applying adjustments for duplicate coordinates
    italian_restaurants_map = mapSetup(df_location) # generating map with restaurant markers

    return italian_restaurants_map

In [34]:
# displaying map with restaurant markers
italian_restaurants_map = get_restaurants_map(top_k_data) # passing the dataframe with restaurants as a result of custom search engine
italian_restaurants_map.save("italian_restaurants_map.html") # saving map in a HTML file
italian_restaurants_map

---
## 5. Advanced Search Engine (Bonus Question)


In [2]:
# setting a dataframe from .tsv file with regional data
data = pd.read_csv('all_restaurants_data_regions.tsv',sep = '\t') 

In this task, we designed an advanced search engine to provide users with a flexible and customizable way to explore Michelin-starred restaurants in Italy. The system allows users to specify detailed search criteria, including restaurant names, cities, and cuisine types, along with additional filtering options such as price ranges, Italian regions, accepted credit cards, and available facilities or services (e.g., Wi-Fi, terrace, air conditioning, parking).

### _Advanced Conjunctive Query_

This step is crucial for building the foundational data structures required for the search engine. The function processes the `restaurantName`, `city`, and `cuisineType` fields in the dataset to create:

1. **Vocabularies**: Maps each unique term in the fields to a unique term ID.
2. **Inverted Indexes**: Maps each term ID to the set of document IDs (restaurants) where the term appears.

In [33]:
def create_vocabularies_and_indexes(data: pd.DataFrame) -> tuple[dict, dict]:
    """
    Creates vocabularies and inverted indexes for specific fields in a dataset.
    
    This function processes the fields `restaurantName`, `city`, and `cuisineType` in the given
    DataFrame to build vocabularies and inverted indexes for each field.
    
    - **Vocabulary**: A dictionary mapping each unique term to a unique term ID for each field.
    - **Inverted Index**: A dictionary mapping each term ID to a set of document IDs (doc_id) where the term appears.
    
    Parameters:
    -----------
    data : pd.DataFrame
        A DataFrame containing restaurant data with columns `restaurantName`, `city`, and `cuisineType`.
    
    Returns:
    --------
    tuple[dict, dict]
        A tuple containing:
        - vocabularies: dict, where each key is a field name (e.g., 'restaurantName') and the value
          is a dictionary mapping terms to unique term IDs.
        - inverted_indexes: dict, where each key is a field name and the value is a dictionary
          mapping term IDs to sets of document IDs containing the term.

    Example:
    --------
    vocabularies, inverted_indexes = create_vocabularies_and_indexes(data)
    """
    vocabularies = {
        'restaurantName': {},
        'city': {},
        'cuisineType': {}
    }
    inverted_indexes = {
        'restaurantName': defaultdict(set),
        'city': defaultdict(set),
        'cuisineType': defaultdict(set)
    }
    term_ids = {
        'restaurantName': 0,
        'city': 0,
        'cuisineType': 0
    }

    # Loop through each restaurant document
    for doc_id, row in data.iterrows():
        for field in ['restaurantName', 'city', 'cuisineType']:
            tokens = preprocess_text(row[field])

            for token in tokens:
                # Assign a new term ID if token not in vocabulary for the field
                if token not in vocabularies[field]:
                    vocabularies[field][token] = term_ids[field]
                    term_ids[field] += 1

                # Add the doc_id to the inverted index for this token in the field
                term_id = vocabularies[field][token]
                inverted_indexes[field][term_id].add(doc_id)

    return vocabularies, inverted_indexes

This step enables the search engine to perform **conjunctive queries**, where only documents containing all the terms in the search query are retrieved. The process involves:

1. **Query Preprocessing**: The input query is tokenized and preprocessed to standardize the text format.
2. **Term Mapping**: Each query term is mapped to its unique term ID using the vocabulary.
3. **Document Retrieval**: The inverted index is used to find documents containing all the query terms. This is achieved by intersecting the document sets associated with each term.

The result is a set of document indices that match all terms in the query, providing precise results for users looking for highly specific matches.

In [32]:
def conjunctive_query_ad(query: str, vocabulary: dict, inverted_index: dict) -> set:
    """
    Performs a conjunctive search query, retrieving documents that contain all terms in the query.

    This function preprocesses the query, converts each term to a term ID using the vocabulary,
    and uses the inverted index to identify documents that contain all specified query terms.

    Parameters:
    -----------
    query : str
        The search query as a string of terms.
    vocabulary : dict
        A dictionary mapping terms to unique term IDs.
    inverted_index : dict
        A dictionary mapping term IDs to sets of document IDs that contain the term.

    Returns:
    --------
    set
        A set of document indices that contain all terms in the query.
    """
    # Preprocess the query text and convert it to term IDs using the vocabulary
    query_tokens = preprocess_text(query)
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]

    # If no terms from the query are found in the vocabulary, return an empty set
    if not query_term_ids:
        return set()

    # Find documents containing all term IDs from the query
    doc_lists = [set(inverted_index[tid]) for tid in query_term_ids if tid in inverted_index]

    # Intersect document lists to get documents that contain all query terms
    matching_docs = set.intersection(*doc_lists) if doc_lists else set()

    # Return the set of matching document indices
    return matching_docs

This step implements a **multi-field conjunctive search** to filter restaurant data based on specific criteria. It allows users to search for restaurants by combining queries for `restaurantName`, `city`, and `cuisineType`. The key features include:

1. **Vocabulary and Inverted Index Creation**: Vocabularies and inverted indexes are created for the specified fields (`restaurantName`, `city`, and `cuisineType`) to enable efficient search.
2. **Field-Specific Search**: Queries are processed for each field, and matching restaurants are identified using a conjunctive query. Only results satisfying all specified criteria are retained.
3. **Filtered Results**: The filtered restaurants are returned in a concise DataFrame containing key details, including the restaurant name, address, cuisine type, price range, and website.

This step enhances the flexibility of the search engine, allowing users to perform detailed and targeted searches across multiple fields simultaneously.

In [31]:
def advanced_field_query(data: pd.DataFrame, restaurant_name_query: str = '', city_query: str = '', cuisine_type_query: str = '') -> pd.DataFrame:
    """
    Performs a multi-field conjunctive search on restaurant data using specified field queries.

    This function creates vocabularies and inverted indexes for specified fields, allowing the user
    to filter restaurant data based on queries for `restaurantName`, `city`, and `cuisineType`.
    It returns only the restaurants that match all specified criteria.

    Parameters:
    -----------
    data : pd.DataFrame
        The dataset containing restaurant information, with relevant columns 'restaurantName', 
        'city', and 'cuisineType'.
    restaurant_name_query : str, optional
        The search query for filtering by restaurant name.
    city_query : str, optional
        The search query for filtering by city.
    cuisine_type_query : str, optional
        The search query for filtering by cuisine type.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the filtered restaurant data, with columns: 
        'restaurantName', 'address', 'cuisineType', 'priceRange', and 'website'.
    """
    # Create vocabularies and inverted indexes for each field
    vocabularies, inverted_indexes = create_vocabularies_and_indexes(data)
    
    # Initialize results as a set of all document indices
    results = set(data.index)

    # Perform search on `restaurantName` if `restaurant_name_query` is provided
    if restaurant_name_query:
        vocabR, invIndexR = vocabularies['restaurantName'], inverted_indexes['restaurantName']
        resultsR = conjunctive_query_ad(restaurant_name_query, vocabR, invIndexR)
        results.intersection_update(resultsR)

    # Perform search on `city` if `city_query` is provided
    if city_query:
        vocabC, invIndexC = vocabularies['city'], inverted_indexes['city']
        resultsC = conjunctive_query_ad(city_query, vocabC, invIndexC)
        results.intersection_update(resultsC)

    # Perform search on `cuisineType` if `cuisine_type_query` is provided
    if cuisine_type_query:
        vocabCT, invIndexCT = vocabularies['cuisineType'], inverted_indexes['cuisineType']
        resultsCT = conjunctive_query_ad(cuisine_type_query, vocabCT, invIndexCT)
        results.intersection_update(resultsCT)

    # Return the filtered data with relevant columns
    filtered_data = data.loc[list(results), ['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    return filtered_data

In this step, we interactively gather search criteria from the user to perform a tailored search for restaurants. Users are prompted to provide inputs for three optional fields: 

1. **Restaurant Name**: Search for specific restaurant names.
2. **City**: Filter restaurants by their city location.
3. **Cuisine Type**: Narrow results to specific types of cuisine.

The provided inputs are passed to the **`advanced_field_query`** function, which processes the queries, filters the restaurant dataset, and returns a DataFrame containing the results. This step ensures the search engine can dynamically respond to user-specific requirements.

In [25]:
# Prompt user for search criteria
restaurant_name_query = input("Enter restaurant name to search: ")
city_query = input("Enter city to search: ")
cuisine_type_query = input("Enter cuisine type to search: ")

# Run the query
results = advanced_field_query(data, restaurant_name_query, city_query, cuisine_type_query)
results

,restaurantName,address,cuisineType,priceRange,website
1152,La Pergola,via Cadlolo 101,"Mediterranean Cuisine, Contemporary",€€€€,https://romecavalieri.com/it/la-pergola-it/
1153,Livello 1,via Duccio di Buoninsegna 25,"Seafood, Modern Cuisine",€€€,http://www.ristorantelivello1.it
263,Antica Pesa,via Garibaldi 18,"Roman, Cuisine from Lazio",€€,https://www.anticapesa.it/
1545,Colline Emiliane,via degli Avignonesi 22,"Emilian, Classic Cuisine",€€,https://www.collineemiliane.com/
394,Osteria Fernanda,via Crescenzo Del Monte 18/24,"Creative, Contemporary",€€€,https://www.osteriafernanda.com
...,...,...,...,...,...
756,Per Me Giulio Terrinoni,vicolo del Malpasso 9,"Italian, Creative",€€€€,https://www.giulioterrinoni.it
502,All'Oro,via Giuseppe Pisanelli 25,"Creative, Italian Contemporary",€€€€,https://www.ristorantealloro.it/
1143,Adelaide,via dell'Arancio 69,Italian Contemporary,€€€,https://www.hotelvilon.com/it/10/ristorante/
1145,Almatò,via Augusto Riboty 20/c,Contemporary,€€€,https://www.almato.it


### _Filtering_

In [27]:
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,text_to_compare,region
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,Air conditioning,"Amex, Dinersclub, Mastercard, Visa",+39 010 247 6191,https://www.ristorante20tregenova.it/,"Farm to table, Modern Cuisine Situated in the ...",Lazio
1,Alessandro Feo,via Angelo Lista 24,Marina di Casal Velino,84040,Italy,€€,"Campanian, Seafood",In a beautiful stone-vaulted building (an old ...,,"Amex, Dinersclub, Mastercard, Visa",+39 328 893 7083,https://www.alessandrofeoristorante.it/,"Campanian, Seafood In a beautiful stone-vaulte...",Lazio
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,12051,Italy,€€,"Piedmontese, Contemporary",This attractive restaurant in the heart of Alb...,"Air conditioning, Terrace, Wheelchair access","Amex, Dinersclub, Maestrocard, Mastercard, Visa",+39 0173 363453,https://www.apewinebar.it/alba/,"Piedmontese, Contemporary This attractive rest...",Lazio
3,Charleston,via Generale Magliocco 19,Palermo,90141,Italy,€€€€,"Modern Cuisine, Creative","Before it became famous in Mondello, the renow...","Air conditioning, Counter dining, Terrace, Whe...","Amex, Mastercard, Visa",+39 091 450171,https://casacharleston.net/,"Modern Cuisine, Creative Before it became famo...",Lazio
4,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,80067,Italy,€€,Seafood,Working in partnership with the nearby fishmon...,"Air conditioning, Terrace","Amex, Dinersclub, Mastercard, Visa",+39 081 1778 3873,https://www.dabobcookfish.com/,Seafood Working in partnership with the nearby...,Lazio


In this step, we enhance the search functionality by applying additional filters to the restaurant data. Users can refine their search results based on the following criteria:

1. **Price Range**: Filter restaurants within specified affordability levels (e.g., €, €€, €€€, €€€€).
2. **Region**: Limit results to specific geographical regions (e.g., Lazio, Tuscany).
3. **Facilities and Services**: Search for restaurants offering specific amenities (e.g., Wi-Fi, Terrace).
4. **Accepted Credit Cards**: Filter by preferred credit card types (e.g., Visa, MasterCard).

The function collects input for these filters, applies them to the previously queried data, and returns a refined DataFrame containing only the relevant results. This allows users to customize their search and discover restaurants that match their exact preferences.

In [ ]:
def filter_results(data: pd.DataFrame) -> pd.DataFrame:
    """
    Applies multiple filters to the restaurant data based on user input.

    This function allows users to filter restaurant data by various criteria, including 
    restaurant name, city, cuisine type, price range, region, facilities, and accepted credit cards.
    It returns a filtered DataFrame with only the relevant columns for display.

    Parameters:
    -----------
    data : pd.DataFrame
        The dataset containing restaurant information with columns for 'restaurantName', 
        'address', 'cuisineType', 'priceRange', 'region', 'facilitiesServices', 'creditCards', and 'website'.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the filtered restaurant data with columns: 
        'restaurantName', 'address', 'cuisineType', 'priceRange', and 'website'.
    """
    # Initial results based on restaurant name, city, and cuisine type queries
    results = advanced_field_query(data, restaurant_name_query, city_query, cuisine_type_query)

    # Get filters from user input
    price_range_filter = input("Enter price ranges to filter, separated by spaces (e.g., €, €€, €€€, €€€€): ").split()
    region_filter = input("Enter regions to filter, separated by spaces: ").split()
    region_filter = [region.title() for region in region_filter]  # Format regions in title case
    facilities_filter = input("Enter facilities/services to filter, separated by spaces: ").split()
    facilities_filter = [facility.title() for facility in facilities_filter]  # Format facilities in title case
    credit_cards_filter = input("Enter accepted credit cards to filter, separated by spaces: ").split()
    credit_cards_filter = [card.title() for card in credit_cards_filter]  # Format credit cards in title case

    # Filter the initial results based on user-defined criteria
    filtered_data = data.loc[results.index, :]
    
    # Apply filters on price range
    if price_range_filter:
        filtered_data = filtered_data[filtered_data['priceRange'].isin(price_range_filter)]
    
    # Apply filters on region
    if region_filter:
        filtered_data = filtered_data[filtered_data['region'].isin(region_filter)]
    
    # Apply filters on facilities/services
    if facilities_filter:
        filtered_data = filtered_data[filtered_data['facilitiesServices'].apply(
            lambda x: all(facility in x for facility in facilities_filter) if isinstance(x, str) else False
        )]
    
    # Apply filters on accepted credit cards
    if credit_cards_filter:
        filtered_data = filtered_data[filtered_data['creditCards'].apply(
            lambda x: any(card in x for card in credit_cards_filter) if isinstance(x, str) else False
        )]

    # Select relevant columns for display
    display_columns = filtered_data[['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    
    return display_columns

In [35]:
filtered_results = filter_results(data)

In [36]:
filtered_results

,restaurantName,address,cuisineType,priceRange,website
805,Hosteria Grappolo d'Oro,piazza della Cancelleria 80,"Roman, Traditional Cuisine",€,https://hosteriagrappolodoro.it/
567,Domenico dal 1968,via Satrico 23,"Roman, Cuisine from Lazio",€,https://www.domenicodal1968.it/
479,Poldo e Gianna Osteria,vicolo Rosini 6/7,"Roman, Traditional Cuisine",€,http://www.poldoegianna.it


Here we implemented the filtering. Price range, region, facilities/services, and accepted credit cards filters were applied sequentially to refine the results further.

---
## 6. Algorithmic Question

### _Algorithm Pseudocode_

We will be focusing only on the core algorithm i.e. the function that will be used to solve the problem and will not show the input procedure in the pseudocode.

```
FUNCTION package_collection(n: Integer, coordinates: List of List of Integers)
    Sort coordinates by x and then y coordinates in ascending order.
    Initialize variable robo_coord as [0,0] which is starting position of robot.
    Initialize variable impossible to False.
    Initlaize variable string to an empty string to store movement sequence.

    For each index in range from 0 to n-1:
    {
        Set variable package_picked to False.

        If the x-coordinate of package at index i is greater than robo_coords' x-coordinate and y-coordinate of package at index i is greater than robo_coord's y-coordinate then:
        {
            Calculate difference between package's and robo-coord's x-coordinate.
            Move robo_coord's x-coordinate to the package's x-coordinate.
            Append 'R' to movement string repeated by the calculated difference.
            Set package_picked to True.
        }

        If the y-coordinate of package at index i is greater than robo_coords' y-coordinate and x-coordinate of package at index i is greater than robo_coord's x-coordinate then:
        {
            Calculate difference between package's and robo-coord's y-coordinate.
            Move robo_coord's y-coordinate to the package's y-coordinate.
            Append 'U' to movement string repeated by the calculated difference.
            Set package_picked to True.
        }

        If package_picked remains False then:
        {
            Set variable impossible to True.
            Exit the for loop.
        }
    }

    Initialize the empty list res.
    If impossible is False then:
        Append 'YES' then append movemeint string to res.
    Else:
        Append 'NO' to res.

    Return res.
END FUNCTION 
```

Now the pseudocode for the sorting algorithm in the first line of the function. We will showcase that using Tim's sort since we will implement it using Python's `sorted()` function.

```
FUNCTION Timsort(coordinates: List of List of Integers)
    Divide the list into small chunks known as run.
    Define a minimum run size, commonly set to 32 or 64.

    For each run of length minimum run in coordinates:
    {
        Sort each element of a run using insertion sort.
    }

    Define merge size as that of minimum run size.
    While merge size is smaller than length of coordinates:
    {
        For each pair of adjacent runs of size merge size:
        {
            Merge the two runs in sorted order using merge sort, comparing first by x-coordinates, and if they are same, then by y-coordinates.
        }

        Double the merge size to merge larger sorted runs in each iteration.
    }

    Return the sorted coordinates list
END FUNCTION
```

### _Algorithm Correctness_

The algorithm is correct based on the following points:
1. We sort the packages by x-coordinates and then by y-coordinates, ensuring we move in lexicographical order.
2. We confirm that each current package's coordinates are not less than the previous one to ensure the robot moves if the path is feasible under the given constraints otherwise we display the the package collection as 'NO' for impossible.
3. We ensure that the robot moves right first when possible and then up ensuring a lexicographical order of path and minimal movement.

To further showcase the accuracy of the algorithm, we have implemented it below.

In [18]:
def package_collection(n: int, coordinates: List[List[int]]):

    # sort the coordinates by x and then y-coordinates
    coordinates= sorted(coordinates, key=lambda x: [x[0], x[1]])
    robo_coord = [0,0]
    impossible = False
    string = ''
    
    for i in range(n):
        
        package_picked = False

        #for right movement
        if coordinates[i][0] > robo_coord[0] and coordinates[i][1] >= robo_coord[1]:
            cnt = coordinates[i][0] - robo_coord[0]
            robo_coord[0] = coordinates[i][0]
            string = string + 'R'*cnt
            package_picked = True 
            
        #for up movement
        if coordinates[i][1] > robo_coord[1] and coordinates[i][0] >= robo_coord[0]:
            cnt = coordinates[i][1] - robo_coord[1]
            robo_coord[1] = coordinates[i][1]
            string = string + 'U'*cnt
            package_picked = True

        #if the package cannot be picked
        if package_picked == False:
            impossible = True
            break
            
    result = [] #appending the result in this list and showcasing it
    if impossible == False:
        result.append('YES')
        result.append(string)
    else:
        result.append('NO')

    return result

In [19]:
# inputting no. of test cases
print("INPUT")
t = int(input('No. of test cases: ')) 

result = list()

for i in range(t):
    coordinates = list()
    #inputting no. of packages
    n = int(input('No. of packages for test case {}: '.format(i+1)))
    for j in range(n):
        #inputting package coordinates
        x_coord, y_coord = map(int, input('Coordinates for package {}: '.format(j+1)).split())
        #storing coordinates of each package in a list
        coordinates.append([x_coord, y_coord])    
    #calling function and storing result
    result.append(package_collection(n, coordinates))

#outputting the result for each test case
print('\nOUTPUT ')
for i in range(t):
    if result[i][0] == 'YES':
        print(result[i][0])
        print(result[i][1])
    else:
        print(result[i][0])

INPUT


No. of test cases:  3
No. of packages for test case 1:  5
Coordinates for package 1:  1 3
Coordinates for package 2:  1 2
Coordinates for package 3:  3 3
Coordinates for package 4:  5 5
Coordinates for package 5:  4 3
No. of packages for test case 2:  2
Coordinates for package 1:  1 0
Coordinates for package 2:  0 1
No. of packages for test case 3:  1
Coordinates for package 1:  4 3



OUTPUT 
YES
RUUURRRRUU
NO
YES
RRRRUUU


### _Time Complexity_

We will be focusing only on the `package_collection` function for computing the time complexity as input is not really the focal area and we want to see the efficiency of our core algorithm.

**1. Sorting Function:**  
We first sort the coordinates using python's `sorted()` function that uses Tim's Sort algorithm. Its time complexity is **_O(nlogn)_**.  
**2. Initializations:**   
We initialze three variables which bring their collecitve time complexity to that of constant i.e. **_O(1)_**.  
**3. For Loop:**    
We iterate through the loop for total no. of n times making the loop's time complexity as **_O(n)_**.  
**4. Inside Loop:**  
Inside the loop, there is a variable initilaztion, three `If` statements within each there are some operations. The time complexity of each of these operations is the same as their collecitve time complexity i.e. **_O(1)_**.  
**5. If-Else Statement:**  
After the loop, there is an `If-Else` statement to append results in the list based on whether the the collection of all packages is possbile or not. Each operation is of constant time complexity bringing the total time complexity to **_O(1)_**.  

##### Summary
- Sorting takes **_O(nlogn)_**.
- The for loop through **n** number of coordinates takes _**O(n)**_.

Therefore the total time complexity of the algorithm is:  
<center><i>O(nlogn)</i> + <i>O(n)</i> = <b></i>O(nlogn)</i></b></center>


### _LLM Computed Time Complexity Comparison_

Following are the time complexity generated by various LLM tools:-  
| LLM Tool | Time Complexity Computed |
|---|---|
| <center>ChatGPT</center> | <center>O(nlogn)</center> |
| <center>Gemini</center> | <center>O(nlogn)</center> |
| <center>Claude AI</center> | <center>O(nlogn)</center> |
| <center>Perplexity</center> | <center>O(nlogn)</center> | 

Each of the LLM tools computed the same time complexity and made similar observations about each part of code while computing the time complexity.

### _Greedy Algorithm & Optimality_

The greedy algorithm is a wonderful approach that guarantees that robot collects all the packages. Moreover, it also provides an optimal path to collect the packages. We will show with an example:  

Let's suppose there are three packages at P1(2,2), P2(0,4) and P3(3,1). The starting position is at R(0,0).  
**Distance from R to P1:** 4 (RRUU)
**Distance from R to P2:** 4 (RRRR)  
**Distance from R to P3:** 4 (RRRU)  
Since all are same, it will choose P2 because it is first in lexicographical order. Now R is at (0,4).  

**Distance from R to P1:** 4 (UULL)  
**Distance from R to P3:** 2 (UL)  
It will choose to go to P3. Now R is at (3,1).  

**Distance from R to P1:** 2 (UL)

Hence the total path is (0,0) -> P2 -> P3 -> P1 i.e. total of 8 moves (RRRRULUL) using greedy approach.  

Now let's take another pathway where we move from (0,0) -> P1 -> P2 -> P3 i.e. total of 10 moves (RRUURRDDUL).

If you see it from any other point of view, the path will either be equal or more than 8 moves. For any other example as well, you will get the greedy approach providing the optimal path.  
However, this is only the case for this particular problem with movement restrictions. If we assign weights to a path or allow diagonal movements, that might make greedy approach provide non-optimal solutions for some cases because greedy algorithm always looks for the shortest path locally instead of finding a globally optimal pathway. 

---